This project will take the top 100 musicians of all time, get the information on their best songs, the artist themselves.
And then put it in a PostGreSQL database

Tracks artists and audio features

In [1]:
#Install the needed libraries
!pip install spotipy
!pip install bs4
!pip install pandas
!pip install selenium
!pip install boto3
!pip install psycopg2

In [2]:
#Import the needed libraries for this project
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from datetime import datetime
from bs4 import BeautifulSoup
from datetime import date
import pandas as pd
import psycopg2
import requests
import json
import boto3
import time
import re

In [3]:
options = Options()
options.add_argument('--headless')
options.add_argument("--window-size=1920,3000")
ser = Service('./chromedriver_win32/chromedriver.exe')
driver = webdriver.Chrome(options=options, service=ser)
driver.get("https://www.thefamouspeople.com/musicians.php")
time.sleep(3)
for i in range(0,5):
    driver.execute_script("window.scrollTo(0,50000)")
    time.sleep(3)
soup = BeautifulSoup(driver.page_source, "html.parser")
print(soup.prettify())

<html class="no-js" lang="en-US">
 <head>
  <script async="" src="https://rules.quantcount.com/rules-p-54Nt-1NAaEEe0.js">
  </script>
  <script defer="" src="https://tagan.adlightning.com/adpushup-thefamouspeople/bl-89d2da9-d9101ae3.js" type="text/javascript">
  </script>
  <script defer="" src="https://tagan.adlightning.com/adpushup-thefamouspeople/b-8db6969-d95c0908.js" type="text/javascript">
  </script>
  <script type="text/javascript">
  </script>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   Famous Musicians - Most Popular Musicians of the World
  </title>
  <link href="//www.thefamouspeople.com/images/favicon_tfp.ico" rel="icon"/>
  <meta content="A comprehensive directory of world's most famous musicians." name="Description"/>
  <meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
  <meta content="on" http-equiv="x-dns-prefetch-control"/>
  <link href="//www.google-analytics.com/" rel="dns-prefetch"/>
  <me

In [4]:
artists_df=pd.DataFrame(columns=['name','birthdate','zodiac','birthplace','isalive','deathdate'])

In [5]:
artist_data=soup.find_all('article', class_='feature col-lg-12 col-md-12 col-sm-12 col-xs-12 eventt internal_space')

for artist in artist_data:
    name=artist.find('a', class_='tileLink').text

    information=artist.find_all('div',class_='desc-q')
    

    birth_date = datetime.strptime(information[1].text.split(":")[1].strip(), "%B %d, %Y").strftime('%Y-%m-%d')
    
    zodiac=information[2].text
    birth_place=information[3].text

    if len(information)==5:
        death_date=datetime.strptime(information[4].text.split(":")[1].strip(), "%B %d, %Y").strftime('%Y-%m-%d')
        is_alive=False
    else:
        is_alive=True
        death_date=''
    artists_df = artists_df.append({'name':name , 'birthdate':birth_date,'zodiac':zodiac.replace("Sun Sign:", "").strip() ,'birthplace':birth_place.replace("Birthplace:", "").strip() ,'isalive':is_alive ,'deathdate':death_date }, ignore_index=True)

artists_df

,name,birthdate,zodiac,birthplace,isalive,deathdate
0,Michael Jackson,1958-08-29,Virgo,"Gary, Indiana, United States",False,2009-06-25
1,Eminem,1972-10-17,Libra,"Saint Joseph, Missouri, United States",True,
2,Taylor Swift,1989-12-13,Sagittarius,"West Reading, Pennsylvania, United States",True,
3,Prince,1958-06-07,Gemini,"Minneapolis, Minnesota, United States",False,2016-04-21
4,Elton John,1947-03-25,Aries,"Pinner, England",True,
...,...,...,...,...,...,...
95,Van Morrison,1945-08-31,Virgo,"Bloomfield, Belfast, Northern Ireland",True,
96,Loretta Lynn,1932-04-14,Aries,"Butcher Hollow, Kentucky, United States",True,
97,David Gilmour,1946-03-06,Pisces,"Cambridge ,England",True,
98,Ennio Morricone,1928-11-10,Scorpio,"Rome, Italy",False,2020-07-06


In [6]:
#Authenticate the Spotify API
CLIENT_ID = '85b72157af5448498f1374633c01162e'
CLIENT_SECRET = '7e871a9f34524944a3f1908549c2d3a6'
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [7]:
artist_urls={}
artist_followers={}
for i, name in enumerate(artists_df['name']):
    artist_search = sp.search(q='artist:' + name, type='artist')
    followers=artist_search['artists']['items'][0]['followers']['total']
    url=artist_search['artists']['items'][0]['external_urls']['spotify'].split('/')[-1]
    artist_urls[name]=url
    artist_followers[name]=followers
    

artists_df['followers'] = artists_df['name'].map(artist_followers)
artists_df['artist_id'] = artists_df['name'].map(artist_urls)

artists_df

,name,birthdate,zodiac,birthplace,isalive,deathdate,followers,artist_id
0,Michael Jackson,1958-08-29,Virgo,"Gary, Indiana, United States",False,2009-06-25,26014639,3fMbdgg4jU18AjLCKBhRSm
1,Eminem,1972-10-17,Libra,"Saint Joseph, Missouri, United States",True,,65989218,7dGJo4pcD2V6oG8kP0tJRR
2,Taylor Swift,1989-12-13,Sagittarius,"West Reading, Pennsylvania, United States",True,,67842068,06HL4z0CvFAxyc27GXpf02
3,Prince,1958-06-07,Gemini,"Minneapolis, Minnesota, United States",False,2016-04-21,7534865,3MHaV05u0io8fQbZ2XPtlC
4,Elton John,1947-03-25,Aries,"Pinner, England",True,,10995681,3PhoLpVuITZKcymswpck5b
...,...,...,...,...,...,...,...,...
95,Van Morrison,1945-08-31,Virgo,"Bloomfield, Belfast, Northern Ireland",True,,2740004,44NX2ffIYHr6D4n7RaZF7A
96,Loretta Lynn,1932-04-14,Aries,"Butcher Hollow, Kentucky, United States",True,,678061,1FE0rls8gfQT3laAeRYNgl
97,David Gilmour,1946-03-06,Pisces,"Cambridge ,England",True,,1690944,2FcC4sDMXme2ziI7tGKMK8
98,Ennio Morricone,1928-11-10,Scorpio,"Rome, Italy",False,2020-07-06,998766,1nIUhcKHnK6iyumRyoV68C


In [8]:
songs_df=pd.DataFrame(columns=['name','artist','danceability','energy','loudness','speechiness','acousticness','instrumentalness','valence','tempo','duration_ms','song_id'])

In [9]:
print("In Progress...")
for i, row  in  artists_df.iterrows():
    artist_search = sp.search(q='artist:' + row['name'], type='artist')
    #url=artist_search['artists']['items'][0]['external_urls']['spotify'].split('/')[-1]
    
    tracks=sp.artist_top_tracks(row["artist_id"],'US')['tracks']
    for j,track in enumerate(tracks):
        uri=track['uri'].split(':')[-1]
        song_name=track['name']
        song_audio=sp.audio_features(tracks=[uri])
        danceability=song_audio[0]['danceability']
        energy=song_audio[0]['energy']
        loudness=song_audio[0]['loudness']
        speechiness=song_audio[0]['speechiness']
        acousticness=song_audio[0]['acousticness']
        instrumentalness=song_audio[0]['instrumentalness']
        valence=song_audio[0]['valence']
        tempo=song_audio[0]['tempo']
        duration_ms=song_audio[0]['duration_ms']
        songs_df=songs_df.append({'name':song_name,'artist':row['name'],'danceability':danceability,'energy':energy,'loudness':loudness,'speechiness':speechiness,'acousticness':acousticness,'instrumentalness':instrumentalness,'valence':valence,'tempo':tempo,'duration_ms':duration_ms,'song_id':uri},ignore_index=True)
    print("{}% Done".format(i+1))
print("Finished!")                            

In Progress...
1% Done
2% Done
3% Done
4% Done
5% Done
6% Done
7% Done
8% Done
9% Done
10% Done
11% Done
12% Done
13% Done
14% Done
15% Done
16% Done
17% Done
18% Done
19% Done
20% Done
21% Done
22% Done
23% Done
24% Done
25% Done
26% Done
27% Done
28% Done
29% Done
30% Done
31% Done
32% Done
33% Done
34% Done
35% Done
36% Done
37% Done
38% Done
39% Done
40% Done
41% Done
42% Done
43% Done
44% Done
45% Done
46% Done
47% Done
48% Done
49% Done
50% Done
51% Done
52% Done
53% Done
54% Done
55% Done
56% Done
57% Done
58% Done
59% Done
60% Done
61% Done
62% Done
63% Done
64% Done
65% Done
66% Done
67% Done
68% Done
69% Done
70% Done
71% Done
72% Done
73% Done
74% Done
75% Done
76% Done
77% Done
78% Done
79% Done
80% Done
81% Done
82% Done
83% Done
84% Done
85% Done
86% Done
87% Done
88% Done
89% Done
90% Done
91% Done
92% Done
93% Done
94% Done
95% Done
96% Done
97% Done
98% Done
99% Done
100% Done
Finished!


In [ ]:
#WebScrape the monthly streams for all the Artists
monthly_listeners={}
for i, row in artists_df.iterrows():
    driver.get('https://open.spotify.com/artist/'+row['artist_id'])
    time.sleep(3)
    monthly_plays=driver.find_element("xpath", "//span[@class='Ydwa1P5GkCggtLlSvphs']")
    monthly_listeners[row['name']]=re.sub("[^0-9]", "",monthly_plays.text)
    print("{}% Done".format(i+1))
artists_df['monthlylisteners'] = artists_df['name'].map(monthly_listeners) 
print("Finished")

1% Done
2% Done
3% Done
4% Done
5% Done
6% Done
7% Done
8% Done
9% Done
10% Done
11% Done
12% Done
13% Done
14% Done
15% Done
16% Done
17% Done
18% Done
19% Done
20% Done
21% Done
22% Done
23% Done
24% Done
25% Done
26% Done
27% Done
28% Done
29% Done
30% Done
31% Done
32% Done
33% Done
34% Done
35% Done
36% Done
37% Done
38% Done
39% Done
40% Done
41% Done
42% Done
43% Done
44% Done
45% Done
46% Done
47% Done
48% Done
49% Done
50% Done
51% Done
52% Done
53% Done
54% Done
55% Done
56% Done
57% Done
58% Done
59% Done
60% Done
61% Done
62% Done
63% Done
64% Done
65% Done
66% Done
67% Done
68% Done
69% Done
70% Done
71% Done
72% Done
73% Done
74% Done
75% Done
76% Done
77% Done
78% Done
79% Done
80% Done
81% Done
82% Done
83% Done
84% Done
85% Done
86% Done
87% Done
88% Done
89% Done


In [ ]:
#Connect to the PostgreSQL database and create a database 'musicpopularity'
try:
    #connect to the default database
    conn = psycopg2.connect(database="postgres", user='postgres', password='Am031749', host='localhost', port= '5432')
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    #create a sparkify database with UTF8 encoding
    cur.execute("DROP DATABASE IF EXISTS musicpopularity;")
    cur.execute("CREATE DATABASE musicpopularity;")
    
    #close default database
    conn.close()
    
    #connect to sparkify database
    conn = psycopg2.connect(database="musicpopularity", user='postgres', password='Am031749', host='localhost', port= '5432')
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    print("Connection Successful")
except psycopg2.Error as e:
    print("Error could not connect to the database")
    print(e)

In [ ]:
#Query for making the artists table
create_artists_table='''
CREATE TABLE artists (
  artist_id TEXT,
  name TEXT NOT NULL,
  birthdate DATE,
  zodiac TEXT,
  birthplace TEXT,
  isalive BOOLEAN NOT NULL,
  deathdate DATE,
  PRIMARY KEY (artist_id)
);
'''
#Query for making the calendar table
create_calendar_table='''
CREATE TABLE calendar (
  date DATE,
  month INTEGER,
  month_name TEXT,
  day_of_month INTEGER,
  day_of_week INTEGER,
  day_name TEXT,
  quarter INTEGER,
  PRIMARY KEY (date)
);
'''
#Query for making the monthlypopularity table
create_monthlypopularity_table="""
CREATE TABLE monthlypopularity (
  date DATE REFERENCES calendar (date),
  artist_id TEXT REFERENCES artists (artist_id),
  followers INTEGER,
  monthlylisteners INTEGER,
  PRIMARY KEY (date, artist_id)
  
);
"""
#Query for making the songs table
create_songs_table="""
CREATE TABLE songs (
  song_id TEXT,
  artist_id TEXT REFERENCES artists (artist_id),
  name TEXT,
  danceability numeric (10,2),
  energy numeric (10,2),
  loudness numeric (10,2),
  speechiness numeric (10,2),
  acousticness numeric (10,2),
  instrumentalness numeric (10,2),
  valence numeric (10,2),
  tempo numeric (10,2),
  duration_ms INTEGER,
  PRIMARY KEY (song_id,artist_id)
);
"""
cur.execute(create_artists_table)
cur.execute(create_calendar_table)
cur.execute(create_monthlypopularity_table)
cur.execute(create_songs_table)
print('Success')

In [ ]:
#Create the dataframe for the songs table that will be uploaded in the database
songstable_df=pd.merge(songs_df, artists_df, left_on='artist', right_on='name')
songstable_df.drop(columns =['artist','name_y','birthdate','isalive','deathdate','followers','monthlylisteners','zodiac','birthplace'], axis=1,inplace=True)
songstable_df.rename(columns={'name_x': 'name'},inplace=True)
songstable_df

In [ ]:
artiststable_df=artists_df.drop(columns=['followers','monthlylisteners'])
artiststable_df

In [ ]:
popularity_df=artists_df.drop(columns=['name','birthdate','zodiac','birthplace','isalive','deathdate'])
popularity_df['date'] =[date.today() for i in range(len(popularity_df))]
#pd.date_range(start=date.today(), periods=len(popularity_df), freq='D')
popularity_df

In [ ]:
#Make the calendar_df that will be loaded into the table 
calendar_list=[[date.today(),date.today().month,date.today().strftime("%B"),date.today().day,date.today().weekday(),date.today().strftime("%A"),((date.today().month)//3)+1]]
calendar_df=pd.DataFrame(calendar_list,columns=['date', 'month', 'month_name', 'day_of_month', 'day_of_week', 'day_name', 'quarter'])
calendar_df

In [ ]:
artists_insert='''
INSERT INTO artists(
    artist_id,
    name,
    birthdate,
    zodiac,
    birthplace,
    isalive,
    deathdate
    ) VALUES (%s,%s,%s,%s,%s,%s,%s);
'''
#Insert the data
for i, row in artiststable_df.iterrows():
    insert_row=list(row)
    name=insert_row[0]
    birthdate=insert_row[1]
    zodiac=insert_row[2]
    birthplace=insert_row[3]
    isalive=insert_row[4]
    deathdate=None if insert_row[5]=="" else insert_row[5]
    artist_id=insert_row[6]
    cur.execute(artists_insert,(artist_id,name,birthdate,zodiac,birthplace,isalive,deathdate))
conn.commit()
print("Success")

In [ ]:
calendar_insert='''
INSERT INTO calendar(
  date,
  month,
  month_name,
  day_of_month,
  day_of_week,
  day_name,
  quarter
    ) VALUES (%s,%s,%s,%s,%s,%s,%s);
'''
#Insert the data
for i, row in calendar_df.iterrows():
    cur.execute(calendar_insert,list(row))
conn.commit()
print("Success")

In [ ]:
popularity_insert='''
INSERT INTO monthlypopularity(
  followers,
  artist_id,
  monthlylisteners,
  date
    ) VALUES (%s,%s,%s,%s);
'''
#Insert the data
for i, row in popularity_df.iterrows():
    cur.execute(popularity_insert,list(row))
conn.commit()
print("Success")

In [ ]:
song_insert='''
INSERT INTO songs(
    name,
    danceability,
    energy,
    loudness,
    speechiness,
    acousticness,
    instrumentalness,
    valence,
    tempo,
    duration_ms,
    song_id,
    artist_id
    ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);
'''
#Insert the data
for i, row in songstable_df.iterrows():
    cur.execute(song_insert,list(row))
conn.commit()
print("Success")